In [1]:
!pip install -q ultralytics
!pip install -q roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.5/644.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.6 MB/s eta 0:00:00


In [2]:
import os
from random import shuffle
from ultralytics import YOLO
from roboflow import Roboflow
import gc
from tqdm.notebook import tqdm
import cv2

In [3]:
rf = Roboflow(api_key="KPdQzbniHwGMSQyhZx76")
project = rf.workspace("surfacedefectdetectiondataset").project("animal-mflmf")
dataset = project.version(1).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.199, to fix: `pip install ultralytics==8.0.134`



Extracting Dataset Version Zip to animal-1 in yolov8:: 100%|██████████| 1536/1536 [00:00<00:00, 5818.38it/s]


In [4]:
def get_batch(path: str, batch_size: int) -> list:
  paths = [i for i in os.listdir(path)]
  shuffle(paths)
  for i in range(0, len(paths), batch_size):
    yield paths[i:min(i + batch_size, len(paths))]


In [5]:
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


In [6]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -q -e .

/content
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 401 (delta 52), reused 44 (delta 44), pack-reused 324
Receiving objects: 100% (401/401), 12.84 MiB | 9.06 MiB/s, done.
Resolving deltas: 100% (205/205), done.
/content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.9 MB/s eta 0:00:00


In [ ]:
!pip uninstall -y supervision
!pip install -q supervision==0.6.0

import supervision as sv
print(sv.__version__)

Found existing installation: supervision 0.6.0
Uninstalling supervision-0.6.0:
  Successfully uninstalled supervision-0.6.0
0.15.0


In [ ]:
!mkdir /content/foundational_output

In [7]:
import torch

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
import os

GROUNDING_DINO_CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(GROUNDING_DINO_CONFIG_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CONFIG_PATH))

/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True


In [ ]:
DATASET_DIR_PATH = f"/content/foundational_output"

In [13]:
%cd {HOME}
!mkdir -p {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

/content
/content/weights


In [14]:
import os

GROUNDING_DINO_CHECKPOINT_PATH = os.path.join(HOME, "weights", "groundingdino_swint_ogc.pth")
print(GROUNDING_DINO_CHECKPOINT_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CHECKPOINT_PATH))

/content/weights/groundingdino_swint_ogc.pth ; exist: True


In [15]:
%cd {HOME}/GroundingDINO

from groundingdino.util.inference import Model

grounding_dino_model = Model(model_config_path=GROUNDING_DINO_CONFIG_PATH, model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH)

/content/GroundingDINO


torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)


final text_encoder_type: bert-base-uncased


In [24]:
file_size_bytes = os.path.getsize(GROUNDING_DINO_CHECKPOINT_PATH)
file_size_mb = file_size_bytes / (1024 * 1024) # Перевод из байтов в мегабайты

print('model size: {:.3f}MB'.format(file_size_mb))

model size: 661.848MB


In [18]:
from typing import List
import cv2

In [19]:
def enhance_class_name(class_names: List[str]) -> List[str]:
    return [
        f"all {class_name}s"
        for class_name
        in class_names
    ]

In [10]:
IMAGES_DIRECTORY = '/content/animal-1/train/images'
IMAGES_EXTENSIONS = ['jpg', 'jpeg', 'png']

CLASSES = ['cat', 'dog']
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.35

In [20]:
import cv2
from tqdm.notebook import tqdm
import time

annotations = {}
batch = get_batch(IMAGES_DIRECTORY, 1000)
images_amount = 0
sum_inference = 0
for image_names in batch:
  for image_name in tqdm(image_names):

      image_path = IMAGES_DIRECTORY + '/' + image_name
      image = cv2.imread(image_path)
      images_amount += 1

      start_time = time.time()
      detections = grounding_dino_model.predict_with_classes(
          image=image,
          classes=enhance_class_name(class_names=CLASSES),
          box_threshold=BOX_TRESHOLD,
          text_threshold=TEXT_TRESHOLD
      )
      end_time = time.time()
      sum_inference += end_time - start_time
      detections = detections[detections.class_id != None]

      annotations[image_name] = detections
  gc.collect()

  0%|          | 0/533 [00:00<?, ?it/s]

The `device` argument is deprecated and will be removed in v5 of Transformers.
None of the inputs have requires_grad=True. Gradients will be None
The `device` argument is deprecated and will be removed in v5 of Transformers.
None of the inputs have requires_grad=True. Gradients will be None


In [21]:
print(f'Avg inference time: {(sum_inference / images_amount * 1000):.4f} ms')

Avg inference time: 350.4298 ms


In [ ]:
sum_of_det = 0
for i in annotations:
  sum_of_det += annotations[i].class_id.shape[0]
sum_of_det

1028

In [ ]:
annotations['cats_10_jpg.rf.3b0875005860118ee6c8badba2eef29e.jpg'].class_id

array([0, 1])

In [ ]:
annotations

In [ ]:
yolo_annot = {}

In [ ]:
for key in annotations:
  yolo_list = []
  print(key)
  img_size = cv2.imread(f'/content/animal-1/train/images/{key}').shape
  for i in range(len(annotations[key].class_id)):
    x_mid = ((annotations[key].xyxy[i][0] + annotations[key].xyxy[i][2]) / 2)/img_size[1]
    y_mid = ((annotations[key].xyxy[i][1] + annotations[key].xyxy[i][3]) / 2)/img_size[0]
    height = (annotations[key].xyxy[i][3] - annotations[key].xyxy[i][1])/img_size[0]
    width = (annotations[key].xyxy[i][2] - annotations[key].xyxy[i][0])/img_size[1]
    det_class = annotations[key].class_id[i]
    yolo_list.append((det_class, x_mid, y_mid, width, height))
  yolo_annot[key] = yolo_list

dog_cats_160_jpg.rf.81fdf77d9f1ad72f7c26fe899ee3f6aa.jpg
dog_cat_641_jpg.rf.b9f689674b894a441d866d2aa9c6391c.jpg
cats_230_jpg.rf.a39bb464e848c8502c551e6b4526f62b.jpg
dog_cat_681_jpg.rf.eb5b5783c7359030a65f38c8a9d20dc8.jpg
dog_cats_138_jpg.rf.25306f493cbfae26dfe77503ea0a52dd.jpg
dog_cat_119_jpg.rf.12e80af94301e63a6d2c504c1c96ba28.jpg
dog_cats_161_jpg.rf.8daa933bb29e031b11abe727b28b5421.jpg
cats_88_jpg.rf.46d3c663cea7a482e3ad022a997539f4.jpg
dog_cat_328_jpg.rf.01e8f263c8dde4d5654d2ef7ecf39a11.jpg
dog_cat_219_jpg.rf.ed8ee63b4672251241b9d34f8b24ac03.jpg
dog_cat_85_jpg.rf.d1bf222adf6db6167c6824d173639d7a.jpg
dog_cats_158_jpg.rf.597a85ae17805b18658cc99f62ef1735.jpg
dog_cats_278_jpg.rf.d301fd83aacdf9dd0244350efb0716bc.jpg
dog_cats_40_jpg.rf.36399957b0bf17bbdc74f58cf7f8bae9.jpg
dog_cat_591_jpg.rf.08b4da974d03c209e92c340408547ee2.jpg
dog_cat_340_jpg.rf.015b7db641d450754a201421b2162a17.jpg
cats_40_jpg.rf.67eff9fe43507047794274d4f2b52b93.jpg
dog_cat_637_jpg.rf.c03244cb2267bb9d6310e067c54ccbab.jpg

In [ ]:
yolo_annot['dog_cat_201_jpg.rf.9c279d2630ce58e0848edcbd8f8d59ab.jpg']

[(0,
  0.6732289314270019,
  0.21040868759155273,
  0.40796737670898436,
  0.4186115264892578)]

In [ ]:
img = cv2.imread('/content/dataset/train-smoke/000007.jpg')

In [ ]:
import numpy as np

In [ ]:
test_file = open('/content/drive/MyDrive/ProjectDrones/labels/003308.txt', 'w+')

In [ ]:
for key in yolo_annot:
  name = '/content/drive/MyDrive/cat_dog_dataset/' + key[:-4] + '.' + 'txt'
  detection_file = open(name, 'w+')
  for detection in yolo_annot[key]:
    det_string = f'{detection[0]} {detection[1]} {detection[2]} {detection[3]} {detection[4]}'
    detection_file.write(det_string + '\n')
  detection_file.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
